In [1]:
import os
import sys
import time
import math
import socket
import shutil
import random
import argparse
import importlib
import data_utils
import scipy.misc
import statistics 
import numpy as np
import pointfly as pf
import tensorflow as tf
from scipy import stats
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

# BASIC CONFIGURATION #

In [2]:
# Append python path
PATH_TRAIN = '../data/modelnet/train_files.txt'
PATH_VALID = '../data/modelnet/test_files.txt'
MODEL_NAME = 'pointcnn_cls'
SETTINGS_FILE = 'modelnet_x3_l4'
DISCARD_NORMAL = True

MODELS = ['../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-08-15-29-50_14671/ckpts/iter-78847',
          '../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-09-07-34-26_9606/ckpts/iter-78847',
          '../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-09-23-39-54_31034/ckpts/iter-78847',
          '../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-10-15-38-57_22597/ckpts/iter-78847',
          '../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-11-07-39-40_11705/ckpts/iter-78847',
          '../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-12-11-10-48_3173/ckpts/iter-78847',
          '../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-13-03-20-06_27936/ckpts/iter-78847',
          '../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-13-19-33-27_20423/ckpts/iter-78847',
          '../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-14-11-40-52_11569/ckpts/iter-78847',
          '../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-15-03-51-29_2197/ckpts/iter-78847',]

# Import model
model = importlib.import_module(MODEL_NAME)

# Import settings
setting_path = os.path.join(str(Path().resolve()), MODEL_NAME)
sys.path.append(setting_path)
setting = importlib.import_module(SETTINGS_FILE)

# List all settings
num_epochs = setting.num_epochs
batch_size = setting.batch_size
sample_num = setting.sample_num
step_val = setting.step_val
rotation_range = setting.rotation_range
rotation_range_val = setting.rotation_range_val
scaling_range = setting.scaling_range
scaling_range_val = setting.scaling_range_val
jitter = setting.jitter
jitter_val = setting.jitter_val
pool_setting_val = None if not hasattr(setting, 'pool_setting_val') else setting.pool_setting_val
pool_setting_train = None if not hasattr(setting, 'pool_setting_train') else setting.pool_setting_train

# BASIC CONFIGURATION

In [3]:
# Prepare inputs
print('{}-Preparing datasets...'.format(datetime.now()))
data_train, label_train, data_val, label_val = setting.load_fn(PATH_TRAIN, PATH_VALID)

# Balance
if setting.balance_fn is not None:
    print('{}-Balancing datasets...'.format(datetime.now()))
    num_train_before_balance = data_train.shape[0]
    repeat_num = setting.balance_fn(label_train)
    data_train = np.repeat(data_train, repeat_num, axis=0)
    label_train = np.repeat(label_train, repeat_num, axis=0)
    data_train, label_train = data_utils.grouped_shuffle([data_train, label_train])
    num_epochs = math.floor(num_epochs * (num_train_before_balance / data_train.shape[0]))

# Save ply
if setting.save_ply_fn is not None:
    print('{}-Saving ply...'.format(datetime.now()))
    folder = os.path.join(root_folder, 'pts')
    print('{}-Saving samples as .ply files to {}...'.format(datetime.now(), folder))
    sample_num_for_ply = min(512, data_train.shape[0])
    if setting.map_fn is None:
        data_sample = data_train[:sample_num_for_ply]
    else:
        data_sample_list = []
        for idx in range(sample_num_for_ply):
            data_sample_list.append(setting.map_fn(data_train[idx], 0)[0])
        data_sample = np.stack(data_sample_list)
    setting.save_ply_fn(data_sample, folder)

if DISCARD_NORMAL:
    data_train = data_train[..., :3]
    data_val = data_val[..., :3]

num_train = data_train.shape[0]
point_num = data_train.shape[1]
num_val = data_val.shape[0]
print('{}-{}/{} training/validation shapes.'.format(datetime.now(), data_train.shape, data_val.shape))

2019-07-18 09:41:12.989211-Preparing datasets...
2019-07-18 09:41:17.392704-(9840, 2048, 3)/(2468, 2048, 3) training/validation shapes.


# PLACEHOLDERS #

In [4]:
# Placeholders
indices = tf.placeholder(tf.int32, shape=(None, None, 2), name="indices")
xforms = tf.placeholder(tf.float32, shape=(None, 3, 3), name="xforms")
rotations = tf.placeholder(tf.float32, shape=(None, 3, 3), name="rotations")
jitter_range = tf.placeholder(tf.float32, shape=(1), name="jitter_range")
global_step = tf.Variable(0, trainable=False, name='global_step')
is_training = tf.placeholder(tf.bool, name='is_training')

data_train_placeholder = tf.placeholder(data_train.dtype, data_train.shape, name='data_train')
label_train_placeholder = tf.placeholder(tf.int64, label_train.shape, name='label_train')
data_val_placeholder = tf.placeholder(data_val.dtype, data_val.shape, name='data_val')
label_val_placeholder = tf.placeholder(tf.int64, label_val.shape, name='label_val')
handle = tf.placeholder(tf.string, shape=[], name='handle')

dataset_train = tf.data.Dataset.from_tensor_slices((data_train_placeholder, label_train_placeholder))
dataset_train = dataset_train.shuffle(buffer_size=batch_size * 4)

if setting.map_fn is not None:
    print('{}-Map function...'.format(datetime.now()))
    dataset_train = dataset_train.map(lambda data, label:
                                      tuple(tf.py_func(setting.map_fn, [data, label], [tf.float32, label.dtype])),
                                      num_parallel_calls=setting.num_parallel_calls)

if setting.keep_remainder:
    print('{}-Keep remainder...'.format(datetime.now()))
    dataset_train = dataset_train.batch(batch_size)
    batch_num_per_epoch = math.ceil(num_train / batch_size)
else:
    print('{}-Dont keep remainder...'.format(datetime.now()))
    dataset_train = dataset_train.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))
    batch_num_per_epoch = math.floor(num_train / batch_size)
dataset_train = dataset_train.repeat(num_epochs)
iterator_train = dataset_train.make_initializable_iterator()
batch_num = batch_num_per_epoch * num_epochs
print('{}-{:d} training batches.'.format(datetime.now(), int(batch_num)))

dataset_val = tf.data.Dataset.from_tensor_slices((data_val_placeholder, label_val_placeholder))
if setting.map_fn is not None:
    dataset_val = dataset_val.map(lambda data, label: tuple(tf.py_func(
        setting.map_fn, [data, label], [tf.float32, label.dtype])), num_parallel_calls=setting.num_parallel_calls)
if setting.keep_remainder:
    dataset_val = dataset_val.batch(batch_size)
    batch_num_val = math.ceil(num_val / batch_size)
else:
    dataset_val = dataset_val.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))
    batch_num_val = math.floor(num_val / batch_size)
iterator_val = dataset_val.make_initializable_iterator()
print('{}-{:d} testing batches per test.'.format(datetime.now(), int(batch_num_val)))

iterator = tf.data.Iterator.from_string_handle(handle, dataset_train.output_types)
(pts_fts, labels) = iterator.get_next()

pts_fts_sampled = tf.gather_nd(pts_fts, indices=indices, name='pts_fts_sampled')
features_augmented = None
if setting.data_dim > 3:
    points_sampled, features_sampled = tf.split(pts_fts_sampled,
                                                [3, setting.data_dim - 3],
                                                axis=-1,
                                                name='split_points_features')
    if setting.use_extra_features:
        if setting.with_normal_feature:
            if setting.data_dim < 6:
                print('Only 3D normals are supported!')
                exit()
            elif setting.data_dim == 6:
                features_augmented = pf.augment(features_sampled, rotations)
            else:
                normals, rest = tf.split(features_sampled, [3, setting.data_dim - 6])
                normals_augmented = pf.augment(normals, rotations)
                features_augmented = tf.concat([normals_augmented, rest], axis=-1)
        else:
            features_augmented = features_sampled
else:
    points_sampled = pts_fts_sampled
points_augmented = pf.augment(points_sampled, xforms, jitter_range)

Instructions for updating:
Colocations handled automatically by placer.
2019-07-18 09:41:23.516817-Keep remainder...
2019-07-18 09:41:23.524713-78848 training batches.
2019-07-18 09:41:23.531856-20 testing batches per test.


# Define model #

In [5]:
net = model.Net(points=points_augmented, features=features_augmented, is_training=is_training, setting=setting)
logits = net.logits
probs = tf.nn.softmax(logits, name='probs')
predictions = tf.argmax(probs, axis=-1, name='predictions')

labels_2d = tf.expand_dims(labels, axis=-1, name='labels_2d')
labels_tile = tf.tile(labels_2d, (1, tf.shape(logits)[1]), name='labels_tile')
loss_op = tf.losses.sparse_softmax_cross_entropy(labels=labels_tile, logits=logits)

with tf.name_scope('metrics'):
    loss_mean_op, loss_mean_update_op = tf.metrics.mean(loss_op)
    print (loss_mean_op, loss_mean_update_op)
    t_1_acc_op, t_1_acc_update_op = tf.metrics.accuracy(labels_tile, predictions)
    print (t_1_acc_op, t_1_acc_update_op)
    t_1_per_class_acc_op, t_1_per_class_acc_update_op = tf.metrics.mean_per_class_accuracy(labels_tile,
                                                                                           predictions,
                                                                                           setting.num_class)
    print (t_1_per_class_acc_op, t_1_per_class_acc_update_op)

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.separable_conv2d instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instruc

# Tensors #

In [6]:
reset_metrics_op = tf.variables_initializer([var for var in tf.local_variables()
                                             if var.name.split('/')[0] == 'metrics'])

_ = tf.summary.scalar('loss/train', tensor=loss_mean_op, collections=['train'])
_ = tf.summary.scalar('t_1_acc/train', tensor=t_1_acc_op, collections=['train'])
_ = tf.summary.scalar('t_1_per_class_acc/train', tensor=t_1_per_class_acc_op, collections=['train'])

_ = tf.summary.scalar('loss/val', tensor=loss_mean_op, collections=['val'])
_ = tf.summary.scalar('t_1_acc/val', tensor=t_1_acc_op, collections=['val'])
_ = tf.summary.scalar('t_1_per_class_acc/val', tensor=t_1_per_class_acc_op, collections=['val'])

lr_exp_op = tf.train.exponential_decay(setting.learning_rate_base, global_step, setting.decay_steps,
                                       setting.decay_rate, staircase=True)
lr_clip_op = tf.maximum(lr_exp_op, setting.learning_rate_min)
_ = tf.summary.scalar('learning_rate', tensor=lr_clip_op, collections=['train'])
reg_loss = setting.weight_decay * tf.losses.get_regularization_loss()
if setting.optimizer == 'adam':
    optimizer = tf.train.AdamOptimizer(learning_rate=lr_clip_op, epsilon=setting.epsilon)
elif setting.optimizer == 'momentum':
    optimizer = tf.train.MomentumOptimizer(learning_rate=lr_clip_op, momentum=setting.momentum, use_nesterov=True)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = optimizer.minimize(loss_op + reg_loss, global_step=global_step)

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

saver = tf.train.Saver(max_to_keep=None)

parameter_num = np.sum([np.prod(v.shape.as_list()) for v in tf.trainable_variables()])
print('{}-Parameter number: {:d}.'.format(datetime.now(), parameter_num))

Instructions for updating:
Use tf.cast instead.
2019-07-18 09:41:32.572309-Parameter number: 599340.


# EVALUATE METHOD

Args:

  **model_path** - path to evaluated model
  
  **num_votes** - how many votes (one vote is one pc rotation & permutation) should be used to eval the model

Returns:
  
  **predictions** - array of the output ofthe classification module with shape: (N, 40), where N is the test clouds len
  
  **true_labels** - true labels of test clouds with the lenght of N
  
  **accuracy** - accuracy of specified model

In [9]:
def evaluate(model_path, num_votes, verbose=False):

    # Start session
    #new_graph = tf.Graph()
    with tf.Session() as sess:

        # Load the model
        saver = tf.train.Saver()
        saver.restore(sess, model_path)
        if verbose:
            print('{}-Checkpoint loaded from {}!'.format(datetime.now(), model_path))

        # Handle
        handle_val = sess.run(iterator_val.string_handle())

        # Data to remember
        voted_logits = []
        voted_labels = None

        # Num votes range
        for _ in range(num_votes):

            # Restart dataset iterator
            sess.run(iterator_val.initializer, feed_dict={
                        data_val_placeholder: data_val,
                        label_val_placeholder: label_val,
                    })

            # Data to remember
            batch_logits = []
            batch_labels = []

            # For each batch in dataset
            for batch_idx_val in range(int(batch_num_val)):

                # Set the batchsize
                if not setting.keep_remainder or num_val % batch_size == 0 or batch_idx_val != batch_num_val - 1:
                    batch_size_val = batch_size
                else:
                    batch_size_val = num_val % batch_size

                # Get the xforms and rotations
                xforms_np, rotations_np = pf.get_xforms(batch_size_val, rotation_range=rotation_range_val,
                                                        scaling_range=scaling_range_val, order=setting.rotation_order)

                # Get logits and labels
                pred_val, labels_val = sess.run([logits, labels_tile], feed_dict={
                             handle: handle_val,
                             indices: pf.get_indices(batch_size_val, sample_num, point_num),
                             xforms: xforms_np,
                             rotations: rotations_np,
                             jitter_range: np.array([jitter_val]),
                             is_training: False,
                         })

                # Remember data
                batch_logits.append(np.squeeze(pred_val, axis=1))
                batch_labels.append(np.squeeze(labels_val))

            # Concatenate
            batch_logits = np.concatenate(batch_logits, axis=0)
            batch_labels = np.concatenate(batch_labels)
            voted_logits.append(batch_logits)
            voted_labels = batch_labels

        # Final concat
        voted_logits = np.sum(np.stack(voted_logits).transpose(1, 2, 0), axis=-1)
        voted_predic = np.argmax(voted_logits, axis=-1)
        voted_accura = float(np.sum(voted_predic == voted_labels)) / len(voted_labels)
        return voted_logits, voted_labels, voted_accura

# # # # # # # # # # # # # # # # # # # # # # # # # # #
# NUM_VOTES TEST
# # # # # # # # # # # # # # # # # # # # # # # # # # #

In [ ]:
NUM_VOTES = 15
RANGE_MODELS = range(10)
num_votes_accs = {i: [] for i in RANGE_MODELS}
for i in num_votes_accs:
    for x in range(1, NUM_VOTES+1):
        _, _, acc = evaluate(MODEL[i], num_votes=x, verbose=False)
        num_votes_accs[i].append(acc)
        print ('i=', i, 'x=', x, 'acc = ', acc)

In [ ]:
for i in num_votes_accs:
    plt.plot(np.arange(1, 1+len(num_votes_accs[i])), num_votes_accs[i])

In [ ]:
num_votes_accs_np = np.zeros((max(RANGE_MODELS), NUM_VOTES), dtype=np.float)
for i in num_votes_accs:
    num_votes_accs_np[i-1] = num_votes_accs[i]
    np.save('log/num_votes_accs.npy', num_votes_accs_np)

# # # # # # # # # # # # # # # # # # # # # # # # #
# MODEL ENSEMBLING 
# # # # # # # # # # # # # # # # # # # # # # # # #

## Calculate probabilities for each test cloud and each model. The output probability array will be the shape of (N, 40, X), where N is the test cloud len and X is the models count to be ensembled.


In [12]:
NUM_MODELS = 10
NUM_VOTES = 12
probabilities = []
true_labels = []
accuracies = []
for x in range(NUM_MODELS):
    pred_vals, true_vals, acc = evaluate(MODELS[x], num_votes=NUM_VOTES, verbose=False)
    probabilities.append(pred_vals)
    accuracies.append(acc)
    true_labels = np.array(true_vals)
    print ('Model =', x, 'acc = ', acc)
    
probabilities = np.stack(probabilities).transpose(1, 2, 0)
accuracies = np.array(accuracies)

INFO:tensorflow:Restoring parameters from ../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-08-15-29-50_14671/ckpts/iter-78847
Model = 0 acc =  0.9165316045380876
INFO:tensorflow:Restoring parameters from ../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-09-07-34-26_9606/ckpts/iter-78847
Model = 1 acc =  0.9149108589951378
INFO:tensorflow:Restoring parameters from ../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-09-23-39-54_31034/ckpts/iter-78847
Model = 2 acc =  0.9173419773095624
INFO:tensorflow:Restoring parameters from ../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-10-15-38-57_22597/ckpts/iter-78847
Model = 3 acc =  0.9145056726094003
INFO:tensorflow:Restoring parameters from ../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-11-07-39-40_11705/ckpts/iter-78847
Model = 4 acc =  0.919773095623987
INFO:tensorflow:Restoring parameters from ../models/cls/pointcnn_cls_modelnet_x3_l4_2019-07-12-11-10-48_3173/ckpts/iter-78847
Model = 5 acc =  0.9145056726094003
INFO:tensorflow:Restoring

In [13]:
np.save('probabilities.npy', probabilities)
np.save('true_labels.npy', true_labels)
np.save('accuracies.npy', accuracies)

# Models evaluation statistics

In [14]:
print('Mean accuracy =', statistics.mean(accuracies))
indices = {}
for k in range(40):
    indices[k] = [i for i, x in enumerate(true_labels) if x == k]
    
validation_max_res = []
test_res_at_validation_max = []

for _ in range(1000):
    validation_indices = []
    test_indices = []
    for k in indices:
        random.shuffle(indices[k])
        split_idx = int(len(indices[k])/2)
        validation_indices += indices[k][:split_idx]
        test_indices += indices[k][split_idx:]
    validation_indices = sorted(validation_indices)
    test_indices = sorted(test_indices)

    validation_true_labels = true_labels[validation_indices]
    validation_probabilities = probabilities[validation_indices]
    test_true_labels = true_labels[test_indices]
    test_probabilities = probabilities[test_indices]

    validation_predictions = np.argmax(validation_probabilities, axis=1)
    validation_compare = np.equal(validation_predictions, np.expand_dims(validation_true_labels, -1))
    validation_accuracies = np.mean(validation_compare, axis=0)

    test_predictions = np.argmax(test_probabilities, axis=1)
    test_compare = np.equal(test_predictions, np.expand_dims(test_true_labels, -1))
    test_accuracies = np.mean(test_compare, axis=0)

    validation_max_res.append(np.max(validation_accuracies))
    test_res_at_validation_max.append(test_accuracies[np.argmax(validation_accuracies)])
    
mean_valid_max = statistics.mean(validation_max_res)
mean_test_at_valid_max = statistics.mean(test_res_at_validation_max)

print('Mean of validation max results =', mean_valid_max)
print('Mean of test result for validation max =', mean_test_at_valid_max)

Mean accuracy = 0.9158833063209076
Mean of validation max results = 0.9214294975688817
Mean of test result for validation max = 0.916790113452188


# Agregate the outputs with sum operation

In [15]:
aggregated_probability = np.sum(probabilities, axis=-1)
aggregated_predictions = np.argmax(aggregated_probability, axis=-1)
float(np.sum(aggregated_predictions == true_labels)) / len(true_labels)

0.9222042139384117

# Agregate the outputs with mean operation

In [16]:
aggregated_probability = np.mean(probabilities, axis=-1)
aggregated_predictions = np.argmax(aggregated_probability, axis=-1)
float(np.sum(aggregated_predictions == true_labels)) / len(true_labels)

0.9222042139384117

# Agregate the outputs with mode operation

In [17]:
aggregated_predictions = np.argmax(probabilities, axis=1)
aggregated_predictions =  np.squeeze(stats.mode(aggregated_predictions, axis=1)[0])
float(np.sum(aggregated_predictions == true_labels)) / len(true_labels)

0.9213938411669368